In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import os
import librosa as lbrs
import wave as wv
import glob
import audioread as ar
import binascii
import sys
import random
from urllib.request import urlretrieve
import os
import tarfile
import pickle
import random
import math

In [0]:
def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

def uncompress(fname):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
    
def indices_to_one_hot(dta):
  one_hot = np.zeros((len(dta), len(np.unique(dta))), np.uint8)
  one_hot[np.arange(len(dta)), dta] = 1
  return one_hot

def read_audio_file(file):
    file_data = np.zeros(sample_length, np.uint8)
    with ar.audio_open(audio_file_path) as audio_file:
      count = 0
      sys.stdout.write('.')
      for buf in audio_file:
        for sample in buf:
          if count >= sample_length:
            break
          file_data[count] = sample
          count = count + 1
        if count >= sample_length:
            break
    return file_data
  
def getbatch(alldata, alllabels, batch_size = 16):
    count = 0
    newdata = alldata.copy()
    arraylength = np.ma.size(newdata, axis=0)
    while count < arraylength/batch_size:
        randstart = random.randint(0, arraylength-batch_size-1)
        count += 1
        x = newdata[randstart:randstart+batch_size]
        yield (x, alllabels[randstart:randstart+batch_size])

In [3]:
download('http://opihi.cs.uvic.ca/sound/genres.tar.gz', 'genres.tar.gz')
if not os.path.isdir('genres'):
  uncompress('genres.tar.gz')
!rm genres.tar.gz

Download file... genres.tar.gz ...
File downloaded


In [26]:
#Alex CNN hyper-parameters
CNN_nb_categories = 10
CNN_nb_files_per_cat = 100
CNN_total_files = CNN_nb_categories * CNN_nb_files_per_cat
CNN_genres_map = {}
CNN_genres_map_reverse = {}

#CNN_noofdatapoints = 10000

CNN_learningrate = 0.001
CNN_nepochs = 10
CNN_batch_size = 5
CNN_noofbatches = 10

#CNN_n_input = 784 # 28x28 image
CNN_n_classes = 10 # 1 for each digit [0-9]
CNN_dropout = 0.75

CNN_data_max_size_div = 1000

#CNN_n_input = (min(map(lambda x:len(x[0]), data)))
CNN_n_input = 1323000//CNN_data_max_size_div
CNN_n_input_sqrt = math.floor(math.sqrt(CNN_n_input))
CNN_n_input_adjusted = CNN_n_input_sqrt**2
CNN_fcl_input = CNN_n_input_sqrt//4
print(CNN_n_input, CNN_n_input_sqrt, CNN_n_input_adjusted, CNN_fcl_input)

1323 36 1296 9


In [0]:
#Alex CNN_utilities_functions
"""
FUNCTION NAME : CNN_find_file
INPUT : number :int, directory :String
OUTPUT : :String
DESCRIPTION : returns the path corresponding to the n-1 element in the directory passed as input
"""
def CNN_find_file(number, directory):
  return(glob.glob(directory)[number-1])

"""
FUNCTION NAME : CNN_one_hot
INPUT : data : list<(list<int>, string)>
OUTPUT : CNN_one_hot : ndarray
DESCRIPTION : create the one-hot matrix for the batch of data
"""
def CNN_one_hot(data):
  nb_files = len(data)
  CNN_one_hot = np.zeros((nb_files, CNN_nb_categories))
  count = 0
  for element in data:
    CNN_one_hot[count, CNN_genres_map_reverse[element[1]]] = 1
    count+=1
  return CNN_one_hot

def CNN_truncate_data(data, devider):
  new_length = len(data)//devider
  return data[0:new_length]

def CNN_normalize_data(data):
  for d in data:
    for e in d:
      e = e/255

In [0]:
#Alex CNN_step_1
"""
FUNCTION NAME : CNN_get_batches
INPUT : CNN_nb_files_per_cat_wanted : int
OUTPUT : CNN_files_to_read : list<String>
DESCRIPTION : returns a list that contains the names of n files per categories, n given as an input
"""
def CNN_get_batches(CNN_nb_files_per_cat_wanted = 20):
  CNN_files_to_read = []
  CNN_count = 0
  CNN_count_file = 0
  
  for CNN_dir_name in glob.glob("genres/*"):
    CNN_genres = CNN_dir_name.split("/")
    CNN_genres = CNN_genres[len(CNN_genres)-1]
    CNN_genres_map[CNN_count] = CNN_genres
    CNN_genres_map_reverse[CNN_genres] = CNN_count
    #Get the batches
    CNN_rand_numbers = [x for x in range(CNN_nb_files_per_cat)]
    random.shuffle(CNN_rand_numbers)
    CNN_rand_numbers = CNN_rand_numbers[0:CNN_nb_files_per_cat_wanted]
    #print(CNN_rand_numbers)
    CNN_f = [CNN_find_file(x, CNN_dir_name + "/*") for x in CNN_rand_numbers]
    CNN_files_to_read.extend(CNN_f)
    #print(CNN_files_to_read, len(CNN_files_to_read))
    #print(len(set(CNN_files_to_read)), len(set(CNN_files_to_read)) == len(CNN_files_to_read))
    """if CNN_count>1:
      break"""
    CNN_count+=1
  return CNN_files_to_read

In [0]:
#Alex CNN_step_2
"""
FUNCTION NAME : CNN_read_batch
INPUT : file_list : list<String>
OUTPUT : CNN_data : ndarray
DESCRIPTION : returns the array containing the data read from the files provided as input
"""
def CNN_read_batch(file_list, devider):
  alldata = []
  for file in file_list:
    #print(file, type(file))
    genre = file.split("/")
    genre = genre[len(genre)-2]
    with ar.audio_open(file) as f:
      data = []
      for buf in f:
        for buf_element in buf:
          data.append(buf_element)
      #print(len(data), type(data),genre)
      data = CNN_truncate_data(data, devider)
      alldata.append((data,genre))
    #break
  return alldata

In [30]:
#Alex CNN_step_3
X = tf.placeholder(tf.float32, [None, CNN_n_input_adjusted])
Y = tf.placeholder(tf.float32, [None, CNN_n_classes])
keep_prob = tf.placeholder(tf.float32)
print(X.shape, Y.shape)

(?, 1296) (?, 10)


In [0]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [0]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [0]:
def conv_net(x, weights, biases, dropout):
    # reshape input to CNN_n_input_sqrtxCNN_n_input_sqrt size
    print(x.shape)
    #x = tf.reshape(x, shape=[-1, CNN_n_input_sqrt, CNN_n_input_sqrt, 1])
    x = tf.reshape(x, shape=[-1, CNN_n_input_sqrt*CNN_n_input_sqrt, 1, 1])
    print(x.shape)
    
    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    #conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    #conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    #fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.reshape(conv1, [-1, weights['wd1'].get_shape().as_list()[0]])
    print(fc1.shape)
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    print(fc1.shape)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [0]:
weights = {
    'wc1': tf.Variable(tf.random_normal([3, 1, 1, 16])),
    #'wc2': tf.Variable(tf.random_normal([3, 1, 1, 64])),
    #'wd1': tf.Variable(tf.random_normal([CNN_fcl_input*CNN_fcl_input*1, 1024])),
    'wd1': tf.Variable(tf.random_normal([CNN_fcl_input*CNN_fcl_input*128, 64])),
    #'out': tf.Variable(tf.random_normal([1024, CNN_n_classes]))
    'out': tf.Variable(tf.random_normal([64, CNN_n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    #'bc2': tf.Variable(tf.random_normal([64])),
    #'bd1': tf.Variable(tf.random_normal([1024])),
    'bd1': tf.Variable(tf.random_normal([64])),
    'out': tf.Variable(tf.random_normal([CNN_n_classes]))
}

In [51]:
# Create the model
model = conv_net(X, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_min = tf.train.AdamOptimizer(learning_rate=CNN_learningrate).minimize(loss)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

(?, 1296)
(?, 1296, 1, 1)
(?, 10368)
(?, 64)
Tensor("Add_14:0", shape=(?, 10), dtype=float32)


In [52]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(CNN_nepochs):
        for _ in range(CNN_noofbatches):
            data = CNN_get_batches(CNN_batch_size)
            data = CNN_read_batch(data, CNN_data_max_size_div)
            batch_y = CNN_one_hot(data)
            batch_x = [x[0] for x in data]
            batch_x = [x[0:CNN_n_input_adjusted] for x in batch_x]
            batch_x = np.array(batch_x)
            batch_y = batch_y.astype(np.float32)
            # Use training data for optimization
            print(batch_x.shape, batch_y.shape)
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y, keep_prob: CNN_dropout})
        # Validate after every epoch
        print("Working up to here!")
        data = CNN_get_batches(CNN_batch_size)
        data = CNN_read_batch(data, CNN_data_max_size_div)
        batch_y = CNN_one_hot(data)
        batch_x = [x[0] for x in data]
        batch_x = [x[0:CNN_n_input_adjusted] for x in batch_x]
        batch_x = np.array(batch_x)
        losscalc, accuracycalc = sess.run([loss, accuracy], 
                                          feed_dict={X:batch_x, Y:batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result
    #TO_DO
    data = CNN_get_batches(CNN_batch_size)
    data = CNN_read_batch(data, CNN_data_max_size_div)
    batch_y = CNN_one_hot(data)
    batch_x = [x[0] for x in data]
    batch_x = [x[0:CNN_n_input_adjusted] for x in batch_x]
    batch_x = np.array(batch_x)
    accuracycalc = sess.run(accuracy, 
                            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
    print("Testing accuracy: %0.4f"%(accuracycalc))

(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Working up to here!
Epoch: 0, Loss: 82983.2266, Accuracy: 0.0800
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Working up to here!
Epoch: 1, Loss: 71239.7578, Accuracy: 0.0800
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Working up to here!
Epoch: 2, Loss: 37230.2461, Accuracy: 0.1200
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Worki

In [60]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(CNN_nepochs):
        for _ in range(CNN_noofbatches):
            data = CNN_get_batches(CNN_batch_size)
            data = CNN_read_batch(data, CNN_data_max_size_div)
            batch_y = CNN_one_hot(data)
            batch_x = [x[0] for x in data]
            batch_x = [x[0:CNN_n_input_adjusted] for x in batch_x]
            CNN_normalize_data(batch_x)
            batch_x = np.array(batch_x)
            batch_y = batch_y.astype(np.float32)
            # Use training data for optimization
            print(batch_x.shape, batch_y.shape)
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y, keep_prob: CNN_dropout})
        # Validate after every epoch
        print("Working up to here!")
        data = CNN_get_batches(CNN_batch_size)
        data = CNN_read_batch(data, CNN_data_max_size_div)
        batch_y = CNN_one_hot(data)
        batch_x = [x[0] for x in data]
        batch_x = [x[0:CNN_n_input_adjusted] for x in batch_x]
        CNN_normalize_data(batch_x)
        batch_x = np.array(batch_x)
        losscalc, accuracycalc = sess.run([loss, accuracy], 
                                          feed_dict={X:batch_x, Y:batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result
    #TO_DO
    data = CNN_get_batches(CNN_batch_size)
    data = CNN_read_batch(data, CNN_data_max_size_div)
    batch_y = CNN_one_hot(data)
    batch_x = [x[0] for x in data]
    batch_x = [x[0:CNN_n_input_adjusted] for x in batch_x]
    CNN_normalize_data(batch_x)
    batch_x = np.array(batch_x)
    accuracycalc = sess.run(accuracy, 
                            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
    print("Testing accuracy: %0.4f"%(accuracycalc))

(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Working up to here!
Epoch: 0, Loss: 99608.3438, Accuracy: 0.1000
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Working up to here!
Epoch: 1, Loss: 91753.4688, Accuracy: 0.1600
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Working up to here!
Epoch: 2, Loss: 59006.1055, Accuracy: 0.0600
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
(50, 1296) (50, 10)
Worki

In [0]:
t = (100,'something')
print(t[0])
print(t[1])
print(CNN_genres_map)
print(CNN_genres_map_reverse)
CNN_one_hot(data)

100
something
{0: 'rock', 1: 'metal', 2: 'hiphop', 3: 'country', 4: 'classical', 5: 'jazz', 6: 'reggae', 7: 'disco', 8: 'pop', 9: 'blues'}
{'rock': 0, 'metal': 1, 'hiphop': 2, 'country': 3, 'classical': 4, 'jazz': 5, 'reggae': 6, 'disco': 7, 'pop': 8, 'blues': 9}


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

In [0]:
print(CNN_n_input)
print(len(data[0][0]))
data_reshaped = [x[0] for x in data]
data_reshaped = [x[0:CNN_n_input] for x in data_reshaped]
data_reshaped = np.array(data_reshaped)

1323008
1323588


In [0]:
print(type(data_reshaped), data_reshaped.shape)

<class 'numpy.ndarray'> (50, 1323008)


In [0]:
f = glob.glob("genres/blues/*")[1]
print(f,type(f))

genres/blues/blues.00064.au <class 'str'>


In [0]:
print(os.path.getsize(f))
f_size = 0
with ar.audio_open(f) as f:
  for buf in f:
    f_size += len(buf)
print(f_size)

1323628
1323588


In [25]:
CNN_n_input_adjusted

121

In [53]:
test = CNN_get_batches(CNN_batch_size)
print(test, len(test))

['genres/rock/rock.00002.au', 'genres/rock/rock.00074.au', 'genres/rock/rock.00081.au', 'genres/rock/rock.00055.au', 'genres/rock/rock.00000.au', 'genres/country/country.00002.au', 'genres/country/country.00040.au', 'genres/country/country.00032.au', 'genres/country/country.00027.au', 'genres/country/country.00065.au', 'genres/blues/blues.00019.au', 'genres/blues/blues.00036.au', 'genres/blues/blues.00018.au', 'genres/blues/blues.00043.au', 'genres/blues/blues.00064.au', 'genres/jazz/jazz.00005.au', 'genres/jazz/jazz.00080.au', 'genres/jazz/jazz.00089.au', 'genres/jazz/jazz.00070.au', 'genres/jazz/jazz.00045.au', 'genres/metal/metal.00023.au', 'genres/metal/metal.00061.au', 'genres/metal/metal.00056.au', 'genres/metal/metal.00078.au', 'genres/metal/metal.00016.au', 'genres/pop/pop.00034.au', 'genres/pop/pop.00005.au', 'genres/pop/pop.00048.au', 'genres/pop/pop.00055.au', 'genres/pop/pop.00064.au', 'genres/reggae/reggae.00041.au', 'genres/reggae/reggae.00056.au', 'genres/reggae/reggae.0

In [0]:
data = CNN_read_batch(test, CNN_data_max_size_div)

In [57]:
for d in data:
  print(min(d[0]))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
